# Lesson

Exploring MCP Primitives: Tools, Resources, and Prompts
Introduction And Lesson Overview

Welcome back! In the previous lesson, you learned what the Model Context Protocol (MCP) is, why it is important, and how to launch a basic MCP server in Python using the FastMCP class. You also explored the two main ways to run your server — using standard input/output (stdio) for local communication and Server-Sent Events (SSE) for networked scenarios. By now, you should feel comfortable starting an MCP server and understanding how it fits into the broader AI integration landscape.

In this lesson, we will build on that foundation by exploring how to define and expose your server’s capabilities. Specifically, you will learn how to add tools, resources, and prompts to your MCP server. These are the core building blocks that allow your server to provide useful functions, share data, and guide AI interactions. By the end of this lesson, you will know how to define each of these primitives and how to interact with them from a client. This will prepare you to create more powerful and interactive MCP servers.
Understanding MCP Server Primitives

To make your MCP server truly useful, you need to expose its capabilities in a way that clients (such as AI agents or other applications) can discover and use. MCP provides three main primitives for this purpose: tools, resources, and prompts.

    Tools are executable functions that perform actions or computations. For example, a tool might add two numbers, fetch data from an API, or process a file. Tools are the “actions” your server can perform on request.
    Resources are data entities that your server exposes. These can be static (like a greeting message or configuration) or dynamic (like user profiles or database records). Resources provide context or information that clients can read and use.
    Prompts are reusable templates that guide AI interactions. They help structure the way an AI model asks questions, explains concepts, or interacts with users. Prompts can accept arguments and generate dynamic messages.

Understanding these primitives is essential because they are the standard way for AI models and clients to discover what your server can do. When you define tools, resources, and prompts, you make your server’s capabilities visible and accessible to the outside world.
Defining MCP Tools

Let’s start by defining a tool. In MCP, a tool is simply a Python function that you expose to clients using a decorator. The FastMCP library makes this easy with the @mcp.tool() decorator. When you decorate a function as a tool, it becomes discoverable and callable by clients.

Here is an example of a simple tool that adds two integers and returns the result:

Python

from mcp.server.fastmcp import FastMCP


mcp = FastMCP(

    name="My Server",

    description="Provides tools, resources, and prompts"

)


# Define a tool that adds two integers and returns the result

@mcp.tool()

def add(a: int, b: int) -> int:

    """Return the sum of a and b."""

    return a + b

In this code, the add function takes two integer arguments, a and b, and returns their sum. The @mcp.tool() decorator registers this function as a tool with the MCP server. You can add as many tools as you like, each with its own logic and documentation.

When a client connects to your server, it can list all available tools and see their names, descriptions, and input parameters. This makes it easy for clients to discover what actions your server can perform.
Exposing MCP Resources

Next, let’s look at resources. Resources are pieces of data that your server exposes for clients to read. Each resource is identified by a unique URI (Uniform Resource Identifier), which acts like an address for that piece of data.

To define a resource in FastMCP, you use the @mcp.resource() decorator and provide a URI. Here is an example:

Python

# Define a resource that returns a static greeting string

@mcp.resource("resource://greeting", description="A simple greeting string returned as plain text.")

def greeting() -> str:

    """A simple greeting string returned as plain text."""

    return "Hello from My Server!"

Notice the description parameter in the decorator. While you might expect FastMCP to use the function’s docstring as the resource description, in practice, the docstring is not always picked up reliably due to current limitations in FastMCP’s docstring parsing. To ensure your resource has a clear, visible description when clients list available resources, you should always provide the description argument explicitly in the decorator.

In this example, the greeting function is registered as a resource with the URI resource://greeting. When a client requests this resource, the server returns the string "Hello from My Server!". Resources can be static, like this greeting, or dynamic, returning different data based on input parameters or server state.

Resources are useful for sharing configuration, documentation, or any other data that clients might need to use as context for AI interactions.
Creating MCP Prompts

Prompts are a unique feature of MCP that allow you to define reusable templates for AI interactions. A prompt is a function that generates a message or instruction, often using input arguments to customize the output.

To define a prompt in FastMCP, use the @mcp.prompt() decorator. Here is an example:

Python

# Define a prompt template that generates a user message about a topic

@mcp.prompt()

def ask_about_topic(topic: str) -> str:

    """Generate a user message asking for an explanation of topic."""

    return f"Can you explain the concept of '{topic}' in simple terms?"

In this code, the ask_about_topic function takes a topic argument and returns a formatted string asking for an explanation of that topic. Prompts like this are helpful for guiding AI models to ask questions, explain concepts, or follow specific workflows.

When a client lists available prompts, it can see their names, descriptions, and required arguments. The client can then request a prompt with specific arguments to generate a customized message.
Connecting to the MCP Server with the MCP Client

To interact with your MCP server, you can use the mcp client interface. The client connects to the server using a transport, such as standard input/output (stdio). Here’s how you can launch the server and establish a connection from a client script:

Python

import asyncio

from mcp import ClientSession, StdioServerParameters

from mcp.client.stdio import stdio_client


async def main():

    # Define server parameters for stdio connection

    server_params = StdioServerParameters(

        command="python",

        args=["mcp_server.py"]

    )


    # Start the stdio client and get the read/write streams for communication

    async with stdio_client(server_params) as (read, write):

        # Create a client session using the communication streams

        async with ClientSession(read, write) as session:

            # Initialize the connection

            await session.initialize()


            # Client-Server interactions go here...


if __name__ == "__main__":

    asyncio.run(main())

This script prepares everything needed to launch your MCP server as a separate process (subprocess) and communicate with it using standard input and output streams (stdio). The process involves several key steps to establish a working connection between the client and the server.

The main steps are:

    Define the server parameters, including the command to run and any arguments (such as the server script filename).
    Use stdio_client to start the server process and obtain the low-level read and write streams for communication.
    Pass these streams to ClientSession, which manages the details of the MCP protocol, including sending requests and receiving responses.
    Call await session.initialize() to perform the initial handshake and set up the session.

By following these steps, you ensure that both the client and server are properly connected and ready to exchange MCP messages. Only after the session is initialized can you safely make requests such as listing tools, reading resources, or calling tools on the server.
Listing Available Tools, Resources, and Prompts

Once the client is connected, it can use the list_tools, list_resources, and list_prompts methods to dynamically discover all the capabilities the server exposes. These methods are important because they allow agents to explore available actions, data, and interaction templates at runtime, enabling flexible and adaptive use of the server’s features. The following code demonstrates how to list each type of primitive and print their names and descriptions, providing a clear overview of what the server offers.

Python

# List all tools the server provides

tools_response = await session.list_tools()

print("Available tools:")

for tool in tools_response.tools:

    print(f" - {tool.name}: {tool.description}")


# List all resources

resources_response = await session.list_resources()

print("\nAvailable resources:")

for res in resources_response.resources:

    print(f" - {res.uri}: {res.description}")


# Introspect prompts

prompts_response = await session.list_prompts()

print("\nAvailable prompts:")

for p in prompts_response.prompts:

    print(f" - {p.name}: {p.description}")

After running this code, you’ll see a summary of all tools, resources, and prompts that the server provides, making it easy to understand what actions and data are accessible:

Plain text

Available tools:

 - add: Return the sum of a and b.


Available resources:

 - resource://greeting: A simple greeting string returned as plain text.


Available prompts:

 - ask_about_topic: Generate a user message asking for an explanation of topic.

Calling a Tool from the Server

After discovering the available tools, you can invoke any tool by name using the call_tool method on the client session. This method allows you to execute server-side functions and retrieve their results by providing the tool’s name and the required arguments as a dictionary.

For example, to call the add tool (which takes two integers and returns their sum), you can use the following code:

Python

# Call the 'add' tool with arguments a=5 and b=3

result_tool = await session.call_tool("add", {"a": 5, "b": 3})

print(f"Result of add tool: {result_tool.content[0].text}")

When you run this code, the client sends a request to the server to execute the add tool with the specified arguments. The server processes the request and returns the result, which you can access from the first element of the returned content list using the .text attribute.

Example output:

Plain text

Result of add tool: 8

This mechanism allows clients to dynamically execute any tool exposed by the server, making it easy to integrate and automate server-side actions from your client applications.
Reading a Resource from the Server

Just as you can call tools by name and pass arguments, you can also access resources exposed by the server using their unique URIs. The example below shows how to read the resource://greeting resource and print its value. Since the server returns a list of resource instances, you access the first element’s .text property to get the actual greeting.

Python

# Fetch the greeting resource

result_resource = await session.read_resource("resource://greeting")

print("Greeting resource:", result_resource.contents[0].text)

When you run this code, the client retrieves the greeting resource and prints its content:

Plain text

Greeting resource: Hello from My Server!

Generating a Prompt with Arguments

Prompts can be used to generate dynamic messages by providing the required arguments. The following code demonstrates how to generate a prompt for a specific topic by calling the prompt with an argument and printing the resulting message. This is especially useful for guiding AI interactions or creating user-facing messages on the fly.

Python

# Fetch and display the formatted prompt for a topic

topic = "machine learning"

result_prompt = await session.get_prompt("ask_about_topic", {"topic": topic})

print(f"Prompt for topic '{topic}': {result_prompt.messages[0].content.text}")

Here, the client sends the topic "machine learning" to the prompt and prints the generated message:

Plain text

Prompt for topic 'machine learning': Can you explain the concept of 'machine learning' in simple terms?

This is the power of MCP: clients can dynamically explore and use whatever tools, resources, and prompts your server provides without needing to know the details in advance.
Summary And Next Steps

In this lesson, you learned how to define and expose the three main MCP server primitives: tools, resources, and prompts. Tools let your server perform actions or computations, resources provide data for clients to use as context, and prompts guide AI interactions with reusable templates. You also saw how a client can connect to your server, discover its capabilities, and interact with each primitive.

These building blocks are at the heart of the MCP framework. By mastering them, you can create servers that are both powerful and flexible, ready to integrate with AI agents and other applications. In the next section, you will get hands-on practice by defining your own tools, resources, and prompts, and by writing client code to interact with them. Happy coding!

# Exercise 1
It’s time to put your knowledge into practice by building a simple client that connects to a basic MCP server.

The MCP server is set up for you and currently has no tools, resources, or prompts—just a name and description. Your task is to complete the main.py file so that it connects to the provided server (mcp_server.py) using stdio.

To complete this exercise, you should:

    Set up the server parameters using StdioServerParameters so the client knows how to launch and connect to mcp_server.py.
    Use the stdio_client to establish a connection to the server.
    Create a client session using the ClientSession class and the communication streams.
    Initialize the session with session.initialize() to perform the handshake and confirm the connection.
    Print a message to show that the connection and initialization were successful.

This exercise will help you become comfortable with the basics of MCP client setup and session initialization.

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # TODO: Define server parameters for stdio connection to mcp_server.py
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"])


    # TODO: Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        # TODO: Create a client session using the communication streams
        async with ClientSession(read, write) as session:
            # TODO: Initialize the connection
            await session.initialize()
            # TODO: Print a success message when connected
            print("Great news, you're connected to the server")

if __name__ == "__main__":
    asyncio.run(main())

# exercise 2

Now that your client can connect to the MCP server, let’s take the next step by adding a tool to the server and making sure your client can discover it.

Your task is to create a tool on the server that multiplies two numbers and returns the result. You’ll also update your client so that it lists all available tools and confirms that your new multiplication tool is visible.

To complete this exercise:

    In mcp_server.py, define a function that multiplies two numbers and returns the product. Be sure to include a clear docstring for your function—MCP uses the function’s docstring as the tool’s description, making it discoverable and understandable to clients.
    Register this function as a tool using the @mcp.tool() decorator so that it is discoverable by clients.
    In main.py, after initializing the session, list all tools provided by the server.
    Print the name and description of each tool to confirm that your multiplication tool appears in the list.

This will help you practice exposing server capabilities and verifying them from the client side.

In [ ]:
# main.py

import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection (perform handshake)
            await session.initialize()

            # TODO: List all tools the server provides
            tools_response = await session.list_tools()
            print("Available tools")
            for tool in tools_response.tools:
                
            # TODO: Print the name and description of each tool
                print(f" - {tool.name}: {tool.description}")
                
if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# mcp server. py

from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    name="My Server",
    description="A simple MCP server with a multiplication tool."
)


# TODO: Define a tool that multiplies two numbers and returns the result
@mcp.tool()
def products(a: float, b: float) -> float:
    """ returns the product of two numbers """
    return a * b
    

if __name__ == "__main__":
    mcp.run(transport="stdio")

# Exercise 3

Now, let’s expand your server’s capabilities by adding a resource and making sure your client can discover it.

Your task is to add a new resource to the server that returns a health status message. This resource should have a unique URI and a clear description. After updating the server, you’ll also update your client so it lists all available resources, showing their URIs and descriptions.

To complete this exercise:

    In mcp_server.py, define a function that returns a health status message, such as "Server is healthy".
    Register this function as a resource using the @mcp.resource() decorator, giving it a unique URI (for example, "resource://health") and a description. (Remember: always provide the description parameter explicitly, since FastMCP does not reliably extract the docstring for resources.)
    In main.py, after listing the tools, add code to list all resources provided by the server.
    Print each resource’s URI and description to confirm your new health resource is visible.

This will help you practice exposing server data and verifying it from the client side, making your MCP server more informative and useful.

In [ ]:
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    name="My Server",
    description="A simple MCP server with a multiplication tool and a health resource."
)


# Existing multiply tool
@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Return the product of a and b."""
    return a * b


# TODO: Define a resource that returns a health status message and register it with a unique URI
@mcp.resource("resource://health" , description="Simple health status message returning plain text when called")
def health_status() -> str:
    """A health status message to state server status"""
    return "Server status is HEALTHY"
    

if __name__ == "__main__":
    mcp.run(transport="stdio")

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection (perform handshake)
            await session.initialize()

            # List all tools the server provides
            tools_response = await session.list_tools()
            print("Available tools:")
            for tool in tools_response.tools:
                print(f" - {tool.name}: {tool.description}")

            # TODO: List all resources the server provides
            resources_response = await session.list_resources()
            print("\nAvailable resources:")
            # TODO: Print each resource's URI and description
            for res in resources_response.resources:
                print(f" - {res.uri}: {res.description}")


if __name__ == "__main__":
    asyncio.run(main())

# Exercise 4
It’s time to explore another MCP primitive, this time you''ll be adding a prompt to the server.

Your task is to create a new prompt on the server that generates a personalized welcome message for a given username. Then, update your client so it can also list all available prompts.

To complete this exercise:

    In mcp_server.py, write a prompt function that takes a username as input and returns a welcome message that includes the username.
    Register this function as a prompt using the @mcp.prompt() decorator.
    In main.py, add code to list all prompts the server provides and print their names and descriptions.

This will help you practice working with prompts and make your MCP server more engaging for users.

In [ ]:
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    name="My Server",
    description="A simple MCP server with a multiplication tool, a health resource, and a welcome prompt."
)


# Existing multiply tool
@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Return the product of a and b."""
    return a * b


# Existing health status resource
@mcp.resource("resource://health", description="Returns the current health status of the server.")
def health_status() -> str:
    """Returns the current health status of the server."""
    return "Server is healthy"


# TODO: Define a prompt function that takes a username and returns a personalized welcome message.
@mcp.prompt()
def ask_username(username: str) -> str:
    """Request username and return a greeting."""
    return f"Welcome '{username}' how are you today?"

if __name__ == "__main__":
    mcp.run(transport="stdio")

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection (perform handshake)
            await session.initialize()

            # List all tools the server provides
            tools_response = await session.list_tools()
            print("Available tools:")
            for tool in tools_response.tools:
                print(f" - {tool.name}: {tool.description}")

            # List all resources the server provides
            resources_response = await session.list_resources()
            print("\nAvailable resources:")
            for res in resources_response.resources:
                print(f" - {res.uri}: {res.description}")

            # TODO: List all prompts the server provides (use session.list_prompts())
            prompts_response = await session.list_prompts()
            print("\nAvailable prompts:")
            
            # TODO: Print each prompt's name and description
            for p in prompts_response.prompts:
                print(f" - {p.name}: {p.description}")
            
if __name__ == "__main__":
    asyncio.run(main())

# Final exercise

You’ve already learned how to connect your client, list server capabilities, and define tools, resources, and prompts. Now it’s time to bring everything together in a single workflow.

Your task is to complete the client so that it not only lists all available tools, resources, and prompts, but also interacts with each one in sequence. After connecting and initializing, your client should:

    Call the multiply tool with two numbers (for example, 6 and 7) and print the result. You can extract the result’s value from a tool call using content[0].text.
    Read the resource://health resource and print its content. You can extract the result’s content from a resource using contents[0].text.
    Generate the welcome_user prompt with a username (such as "Alice") and print the resulting message. You can extract the result’s message from a prompt using messages[0].content.text.

Add your code in the marked sections so that each server response is printed clearly. This exercise will help you see how all the MCP primitives work together in a real client workflow.

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection (perform handshake)
            await session.initialize()

            # TODO: Call the multiply tool with two numbers (e.g., 6 and 7) and print the result
            result_tool = await session.call_tool("multiply", {"a": 7, "b": 23})
            print(f"Result of multiply tool: {result_tool.content[0].text}")
            
            # TODO: Read the health resource and print its content
            result_resource = await session.read_resource("resource://health")
            print("Health Status resource:", result_resource.contents[0].text)

            # TODO: Generate the welcome prompt for a username (e.g., "Alice") and print the result
            uname = "Quimbano Prime"
            result_prompt = await session.get_prompt("welcome_user", {"username": uname})
            print(f"Welcome username: '{uname}': {result_prompt.messages[0].content.text}")

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    name="My Server",
    description="A simple MCP server with a multiplication tool, a health resource, and a welcome prompt."
)


# Multiply tool
@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Return the product of a and b."""
    return a * b


# Health status resource
@mcp.resource("resource://health", description="Returns the current health status of the server.")
def health_status() -> str:
    """Returns the current health status of the server."""
    return "Server is healthy"


# Welcome prompt
@mcp.prompt()
def welcome_user(username: str) -> str:
    """Generate a personalized welcome message for the given username."""
    return f"Welcome, {username}! We're glad to have you here."


if __name__ == "__main__":
    mcp.run(transport="stdio")

# Exercise 1

You’ve just seen how to set up an MCP server and learned how to expose service actions as tools with clear documentation. Now it’s your turn to put this into practice.

Your task is to set up the shopping list service inside the MCP server file and create a tool for adding new items. Follow these steps:

    Initialize the ShoppingListService so it’s ready to use.
    Define the add_item function and register it as an MCP tool. The function should take two parameters: name (a string representing the item's name) and quantity (an integer representing how many to add).
    Write a clear docstring for the tool. Be sure to explain what the tool does, describe its parameters and state what it returns (the new item’s ID).
    In the function, call the shopping list service to add the item and return the new item’s ID.

When the server starts, it should print out all registered tools so you can see that your tool is discoverable.

In [ ]:
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# TODO: Initialize the shopping list service
shopping_list = ShoppingListService()

# TODO: Define the add_item function and register it as an MCP tool
@mcp.tool()
def add_item(name: str, quantity: int) -> str:

    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.
      
    Returns:
      str: The unique ID of the newly added item.
    """

    return shopping_list.add_item(name, quantity)
    # TODO: Parameters: name (str), quantity (int)
    # TODO: Write a docstring that explains what this tool does, its parameters, and what it returns
    # TODO: Call the shopping list service to add the item and return the new item's ID


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

In [ ]:
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # List all tools
            tools_list = await session.list_tools()
            
            # Display each tool's name, description and input schema
            print("Available tools:\n")
            for tool in tools_list.tools:
                print(f"Name: {tool.name}")
                print(f"Description: {tool.description}")
                print(f"Input Schema:\n{tool.inputSchema}")
            
if __name__ == "__main__":
    asyncio.run(main())

# Exercise 2

You’ve just seen how to set up an MCP server and create a tool for adding items to the shopping list. Now it’s time to try out the client side and see how to actually use this tool in practice.

Your task is to complete the client code so it connects to the MCP server and calls the add_item tool. Here’s what you need to do:

    Call the add_item tool with a sample name and quantity (for example, "Bananas" and 3).
    Capture the response from the tool call.
    Extract the ID of the newly added item from the response.
    Print the added item’s ID so you can see that it worked.

This exercise will help you understand how to interact with MCP tools from a client and how to handle the results.

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # TODO: Call the add_item tool with sample parameters (e.g., name="Bananas", quantity=3)
            response_add = await session.call_tool(
                "add_item",
                {
                    "name": "Bananas",
                    "quantity" : 3
                }
            )

            # TODO: Extract the ID of the newly added item from the response's content[0].text
            added_id = response_add.content[0].text
            # TODO: Print the added item's ID
            print(f"Added item ID: {added_id}")

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

In [ ]:
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

# Exercise 3
Now it’s time to expand your server’s abilities by making it possible to remove items and mark them as purchased. Your task is to add two new tools to the MCP server:

    remove_item: This tool should allow clients to remove an item from the shopping list by its unique ID.
    mark_purchased: This tool should allow clients to mark an item as purchased or not, using the item’s ID and an optional boolean flag (purchased, which defaults to True).

For each tool, be sure to:

    Use the correct method from the ShoppingListService.
    Write a clear docstring that explains what the tool does, describes its parameters, and states what it returns.
    Make sure the tool returns a boolean value to indicate whether the operation was successful.

When you’re done, the server should print out all registered tools, including your new ones. This exercise will help you practice adding new features and writing helpful documentation for your tools.

In [ ]:
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


# TODO: Define the remove_item tool
#   - It should take an item_id parameter (string)
#   - It should return a boolean value
#   - Write a docstring that explains what this tool does, its parameter, and what it returns
#   - Call the shopping_list.remove_item() method
@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.
    
    Parameters:
      item_id (str): The unique ID of the item.
      
    Returns:
      bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)

# TODO: Define the mark_purchased tool
#   - It should take an item_id parameter (string) and an optional purchased parameter (bool, default True)
#   - It should return a boolean value
#   - Write a docstring that explains what this tool does, its parameters, and what it returns
#   - Call the shopping_list.set_purchased() method
@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.

    Parameters:
      item_id (str): The unique ID of the item.
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.

    Returns:
      bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)

if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

In [ ]:
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # List all tools
            tools_list = await session.list_tools()
            
            # Display each tool's name, description and input schema
            print("Available tools:\n")
            for tool in tools_list.tools:
                print(f"Name: {tool.name}")
                print(f"Description: {tool.description}")
                print(f"Input Schema:\n{tool.inputSchema}")
                print("-" * 150)
            
if __name__ == "__main__":
    asyncio.run(main())

# Exercise 4
Tools for adding, removing, and updating shopping list items are in place—now it’s time to boost your server by enabling clients to fetch the current list, with an optional filter for purchased status.

Your task is to implement a new MCP tool called fetch_items in your server. This tool should:

    Accept an optional parameter called purchased (a boolean, defaulting to None).
    Return a list of shopping items, filtered by the purchased status if provided.
    Include a clear docstring that explains what the tool does, describes its parameter, and states what it returns.
    Call the get_items method from the ShoppingListService to retrieve the data.

Once you’ve added the tool, update your client code to:

    Call the fetch_items tool with no filter (to get all items) and print each item in the result.
    Optionally, try calling the tool with purchased=True and purchased=False to see how filtering works.

This exercise will help you practice exposing dynamic data through tools and handling optional parameters in your MCP server.

In [ ]:
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.

    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.

    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.

    Parameters:
      item_id (str): The unique ID of the item.

    Returns:
      bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)


@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.

    Parameters:
      item_id (str): The unique ID of the item.
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.

    Returns:
      bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)


# TODO: Implement the fetch_items tool
#   - Use the @mcp.tool() decorator
#   - The function should be called fetch_items and take an optional purchased parameter (bool, default None)
#   - The return type should be list
#   - Write a docstring that explains what the tool does, its parameter, and what it returns
#   - Call shopping_list.get_items(purchased) and return the result
@mcp.tool()
def fetch_items(purchased: bool = True) -> list:
    """
    Get all items with the option to filter by purchase status - true or false

    Parameters:
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.

    Returns:
      list of items: filtered to purchased if purchased option is true; otherwise list of all items regardless of purchase status if purchase is False.
    """
    return shopping_list.get_items(purchased)

if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio")

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    # Define server parameters for a stdio connection
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    # Establish stdio client connection
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # TODO: Call the fetch_items tool with no filter (get all items)
            response_fetch = await session.call_tool("fetch_items", {})
            # TODO: Print each item in the returned list
            print("Current shopping list items:")
            for item in response_fetch.content:
                print(item.text)

            # TODO (optional): Try calling fetch_items with purchased=True and print the results
            response_fetch_purchased = await session.call_tool("fetch_items", 
            {
                "purchased" : True
            })
            # TODO: Print each item in the returned list
            print("Current purchased= True shopping list items:")
            for item in response_fetch_purchased.content:
                print(item.text)
            # TODO (optional): Try calling fetch_items with purchased=False and print the results
            response_fetch_purchased_false = await session.call_tool("fetch_items", 
            {
                "purchased" : False
            })
            # TODO: Print each item in the returned list
            print("Current purchased=False shopping list items:")
            for item in response_fetch_purchased_false.content:
                print(item.text)
if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False

# Final Exercise

It’s time to bring everything together and test your server in a real scenario.

Your task is to write a client script that exercises all four tools — add_item, mark_purchased, remove_item, and fetch_items — in a complete test sequence. This will help you see how the tools work together and ensure your server is behaving as expected.

Here’s what you should do:

    Call the add_item tool to add "Carrots" with quantity 4.
    Save the ID of the newly added item in a variable called added_id.
    Print the added item's ID.
    Call the mark_purchased tool using added_id to mark the new item as purchased.
    Print whether the operation was successful.
    Call the remove_item tool using added_id to remove the new item.
    Print whether the removal was successful.

This exercise will help you practice integrating multiple server actions and give you confidence that your tools work well together.

In [ ]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client


async def main():
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"]
    )

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            print("=== Initial shopping list ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(item.text)

            print("\n=== Adding a new item: Carrots (4) ===")
            # TODO: Call the add_item tool to add "Carrots" with quantity 4
            response_add = await session.call_tool(
                "add_item",
                {
                    "name": "Carrots",
                    "quantity" : 4
                }
            )

            # TODO: Save the ID of the newly added item in a variable called added_id
            added_id = response_add.content[0].text
            # TODO: Print the added item's ID
            print(f"Added ID item: {added_id}")

            print("\n=== Shopping list after adding ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(item.text)

            print("\n=== Marking the new item as purchased ===")
            # TODO: Call the mark_purchased tool using added_id
            response = await session.call_tool(
                "mark_purchased",
                {
                    "item_id" : added_id,
                    "purchased": True
                }
            )
            # TODO: Print whether the operation was successful
            for item in response.content:
                print(item.text)

            print("\n=== Shopping list after marking as purchased ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(item.text)

            print("\n=== Removing the new item ===")
            # TODO: Call the remove_item tool using added_id
            response = await session.call_tool("remove_item", {"item_id": added_id})
            
            # TODO: Print whether the removal was successful
            for item in response.content:
                print(item.text)

            print("\n=== Final shopping list ===")
            response = await session.call_tool("fetch_items", {})
            for item in response.content:
                print(item.text)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items"
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.
    
    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.
      
    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.
    
    Parameters:
      item_id (str): The unique ID of the item.
      
    Returns:
      bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)


@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.
    
    Parameters:
      item_id (str): The unique ID of the item.
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.
      
    Returns:
      bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)


@mcp.tool()
def fetch_items(purchased: bool = None) -> list:
    """
    Retrieve all shopping list items, optionally filtered by purchased status.
    
    Parameters:
        purchased (bool, optional): Filter items by purchased status.
            None returns all items,
            True returns only purchased items,
            False returns only unpurchased items.
    
    Returns:
        list: A list of shopping items matching the filter criteria.
    """
    return shopping_list.get_items(purchased)


if __name__ == "__main__":
    # Run the server with the stdio transport
    mcp.run(transport="stdio") 

In [ ]:
import uuid


class ShoppingListService:
    """Manages a collection of shopping items with basic operations."""
    
    def __init__(self):
        self.items = [
            {"id": str(uuid.uuid4()), "name": "Milk", "quantity": 2, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Bread", "quantity": 1, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Eggs", "quantity": 12, "purchased": True},
            {"id": str(uuid.uuid4()), "name": "Apples", "quantity": 6, "purchased": False},
            {"id": str(uuid.uuid4()), "name": "Coffee", "quantity": 1, "purchased": False}
        ]
    
    def get_items(self, purchased=None):
        """Get all items, optionally filtered by completion status."""
        if purchased is None:
            return self.items.copy()
        
        return [item for item in self.items if item['purchased'] == purchased]
    
    def add_item(self, name, quantity):
        """Add a new item to the shopping list and return its ID."""
        # Generate a unique ID for the new item
        new_item_id = str(uuid.uuid4())
        
        # Create the new item as a dictionary
        new_item = {
            'id': new_item_id,
            'name': name,
            'quantity': quantity,
            'purchased': False
        }
        
        # Add the item to the list
        self.items.append(new_item)
        
        # Return the ID of the new item
        return new_item_id
    
    def remove_item(self, item_id):
        """Remove an item from the shopping list by its ID."""
        # Find the index of the item with matching ID
        for index, item in enumerate(self.items):
            if item['id'] == item_id:
                # Remove the item when found
                del self.items[index]
                return True
        
        # Return False if no item was found with that ID
        return False
    
    def set_purchased(self, item_id, purchased=True):
        """Mark an item as purchased or not purchased by its ID."""
        # Look for the item with the matching ID
        for item in self.items:
            if item['id'] == item_id:
                # Update the item's purchased status
                item['purchased'] = purchased
                return True
        
        # Return False if no item was found with that ID
        return False